<a href="https://colab.research.google.com/github/jakob-ra/relation_extraction_firm_alliances/blob/main/Firm_relation_extraction_LUKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-f44f9980-734c-78db-edd1-e2034b5048a5)


In this notebook, we are going to fine-tune [`LukeForEntityPairClassification`](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) on a supervised **relation extraction** dataset.

The goal for the model is to predict, given a sentence and the character spans of two entities within the sentence, the relationship between the entities.

* Paper: https://arxiv.org/abs/2010.01057
* Original repository: https://github.com/studio-ousia/luke

In [3]:
!pip install -q transformers 

In [4]:
!pip install -q pytorch-lightning wandb

## Read in data

Let's download the data from the web, hosted on Dropbox.

Each row in the dataframe consists of a news article. 

In [5]:
import pandas as pd

df = pd.read_pickle("https://www.dropbox.com/s/j6jvmzpedgf7jxg/kb_6class_unbalanced_neg_examples.pkl?dl=1")
df.head()

,document,entities,entity_spans,relation
0,Salix Pharmaceuticals Ltd (SP) and Pharmatel P...,"[Salix Pharmaceuticals Ltd, Pharmatel Pty Ltd]","[(0, 25), (35, 52)]","[Marketing, StrategicAlliance]"
2,Praxair Inc and Phillips 66 Co formed a strate...,"[Praxair Inc, Phillips 66 Co]","[(0, 11), (16, 30)]",[StrategicAlliance]
3,Apple Computer Inc (ACI) and Samsung Electroni...,"[Apple Computer Inc, Samsung Electronics Co Ltd]","[(0, 18), (29, 55)]",[StrategicAlliance]
4,Robert Koch Institute granted Hoechst Schering...,"[Robert Koch Institute, Hoechst Schering AgrEv...","[(0, 21), (30, 58)]","[Licensing, ResearchandDevelopment, StrategicA..."
5,Thai Advance Innovation Co Ltd and Ai And Robo...,"[Thai Advance Innovation Co, Ai And Robotics V...","[(0, 26), (35, 62)]",[JointVenture]


In [6]:
df.entity_spans.iloc[0]

[(0, 25), (35, 52)]

This is the frequency of our relation types:



In [7]:
df.relation.explode().value_counts()

StrategicAlliance         34707
JointVenture               9074
Marketing                  7267
Licensing                  5170
Manufacturing              5125
ResearchandDevelopment     4936
Name: relation, dtype: int64

Let's one hot encode them.

In [8]:
rel_label_names = df.relation.explode().value_counts().index.to_list()
def label2ids(labels, rel_label_names=rel_label_names):
  return [1 if rel_name in labels else 0 for rel_name in rel_label_names]
df['rel_one_hot'] = df.relation.apply(label2ids)
df.head()

,document,entities,entity_spans,relation,rel_one_hot
0,Salix Pharmaceuticals Ltd (SP) and Pharmatel P...,"[Salix Pharmaceuticals Ltd, Pharmatel Pty Ltd]","[(0, 25), (35, 52)]","[Marketing, StrategicAlliance]","[1, 0, 1, 0, 0, 0]"
2,Praxair Inc and Phillips 66 Co formed a strate...,"[Praxair Inc, Phillips 66 Co]","[(0, 11), (16, 30)]",[StrategicAlliance],"[1, 0, 0, 0, 0, 0]"
3,Apple Computer Inc (ACI) and Samsung Electroni...,"[Apple Computer Inc, Samsung Electronics Co Ltd]","[(0, 18), (29, 55)]",[StrategicAlliance],"[1, 0, 0, 0, 0, 0]"
4,Robert Koch Institute granted Hoechst Schering...,"[Robert Koch Institute, Hoechst Schering AgrEv...","[(0, 21), (30, 58)]","[Licensing, ResearchandDevelopment, StrategicA...","[1, 0, 0, 1, 0, 1]"
5,Thai Advance Innovation Co Ltd and Ai And Robo...,"[Thai Advance Innovation Co, Ai And Robotics V...","[(0, 26), (35, 62)]",[JointVenture],"[0, 1, 0, 0, 0, 0]"


In [9]:
def ids2labels(ids, rel_label_names=rel_label_names):
  labels = []
  for idx, label in enumerate(rel_label_names):
    if ids[idx] == 1:
      labels.append(label)
    
  return labels

In [10]:
df.shape

(46617, 5)

## Define the PyTorch dataset and dataloaders

Next, we define regular PyTorch datasets and corresponding dataloaders. In PyTorch, you need to define a `Dataset` class that inherits from `torch.utils.data.Dataset`, and you need to implement 3 methods: the `init` method (for initializing the dataset with data), the `len` method (which returns the number of elements in the dataset) and the `getitem()` method, which returns a single item from the dataset.

In our case, each item of the dataset consists of a sentence, the spans of 2 entities in the sentence, and a label of the relationship. We use `LukeTokenizer` (available in the Transformers library) to turn these into the inputs expected by the model, which are `input_ids`, `entity_ids`, `attention_mask`, `entity_attention_mask` and `entity_position_ids`.

For more information regarding these inputs, refer to the [docs](https://huggingface.co/transformers/model_doc/luke.html#lukeforentitypairclassification) of `LukeForEntityPairClassification`.


In [11]:
from transformers import LukeTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

tokenizer = LukeTokenizer.from_pretrained("studio-ousia/luke-base", task="entity_pair_classification")

class RelationExtractionDataset(Dataset):
    """Relation extraction dataset."""

    def __init__(self, data):
        """
        Args:
            data : Pandas dataframe.
        """
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]

        text = item.document

        encoding = tokenizer(text, entity_spans=item.entity_spans, padding="max_length", truncation=True, return_tensors="pt")

        for k,v in encoding.items():
          encoding[k] = encoding[k].squeeze()

        encoding["label"] = torch.tensor(item.rel_one_hot)

        return encoding

Here we instantiate the class defined above with 3 objects: a training dataset, a validation dataset and a test set.

In [12]:
from sklearn.model_selection import train_test_split


df = df.sample(1000)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=False)

print("FULL Dataset: {}".format(len(df)))
print("TRAIN Dataset: {}".format(len(train_df)))
print("TEST Dataset: {}".format(len(test_df)))
print("VALIDATION Dataset: {}".format(len(val_df)))

# define the dataset
train_dataset = RelationExtractionDataset(data=train_df)
valid_dataset = RelationExtractionDataset(data=val_df)
test_dataset = RelationExtractionDataset(data=test_df)

FULL Dataset: 1000
TRAIN Dataset: 640
TEST Dataset: 200
VALIDATION Dataset: 160


In [13]:
train_dataset[0].keys()

dict_keys(['input_ids', 'entity_ids', 'entity_position_ids', 'attention_mask', 'entity_attention_mask', 'label'])

Let's define the corresponding dataloaders (which allow us to iterate over the elements of the dataset):

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=2)
test_dataloader = DataLoader(test_dataset, batch_size=2)

Let's verify an example of a batch:

In [15]:
batch = next(iter(train_dataloader))
tokenizer.decode(batch["input_ids"][1])

'<s> <ent> University of Cebu <ent>  and <ent2>  Cisco Systems Philippines <ent2>, a unit of Cisco Systems Inc, formed a strategic alliance to provide information technology education services in the Philippines. The hands-on curriculum taught students to design, build and maintain computer networks. Graduates of the course got globally-accredited Cisconetworking certificate.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [16]:
ids2labels(batch["label"][0])

['StrategicAlliance']

## Define a PyTorch LightningModule

Let's define the model as a PyTorch LightningModule. A `LightningModule` is actually an `nn.Module`, but with some extra functionality.

For more information regarding how to define this, see the [docs](https://pytorch-lightning.readthedocs.io/en/latest/?_ga=2.56317931.1395871250.1622709933-1738348008.1615553774) of PyTorch Lightning.

In [17]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
from transformers import LukeForEntityPairClassification, AdamW
import pytorch_lightning as pl
from sklearn import metrics


threshold = 0.5

class LUKE(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = LukeForEntityPairClassification.from_pretrained("studio-ousia/luke-base", num_labels=len(rel_label_names))

    def forward(self, input_ids, entity_ids, entity_position_ids, attention_mask, entity_attention_mask):     
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, entity_ids=entity_ids, 
                             entity_attention_mask=entity_attention_mask, entity_position_ids=entity_position_ids)
        return outputs
    
    def common_step(self, batch, batch_idx):
        labels = batch['label'].float()
        del batch['label']
        outputs = self(**batch)
        logits = outputs.logits

        criterion = torch.nn.BCEWithLogitsLoss() # multi-label classification
        loss = criterion(logits, labels)
        predictions = (torch.sigmoid(logits)>threshold).float()

        correct = (predictions == labels).sum().item()
        accuracy = correct/batch['input_ids'].shape[0]
        
        # targets = labels.cpu().detach().numpy().tolist()
        # outputs = predictions.cpu().detach().numpy().tolist()
        # with torch.no_grad():
        #   f1_scores_micro, f1_scores_macro = [], []
        #   for target, output in zip(targets, outputs):
        #     print('Target: ', target, ' Output: ', output)
        #     f1_scores_micro.append(metrics.f1_score(targets, outputs, average='micro'))
        #     f1_scores_macro.append(metrics.f1_score(targets, outputs, average='macro'))
        #   f1_score_micro = sum(f1_scores_micro)/len(f1_scores_micro)
        #   f1_score_macro = sum(f1_scores_macro)/len(f1_scores_macro)
        # self.log("training_f1_score_macro", f1_score_macro, on_epoch=True)

        return loss, accuracy
      
    def training_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)     
        # logs metrics for each training_step,
        # and the average across the epoch
        self.log("training_loss", loss)
        self.log("training_accuracy", accuracy)

        return loss

    def validation_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)     
        self.log("validation_loss", loss, on_epoch=True)
        self.log("validation_accuracy", accuracy, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss, accuracy = self.common_step(batch, batch_idx)     

        return loss

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=5e-5)
        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader


model = LUKE()

Some weights of the model checkpoint at studio-ousia/luke-base were not used when initializing LukeForEntityPairClassification: ['embeddings.position_ids']
- This IS expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LukeForEntityPairClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LukeForEntityPairClassification were not initialized from the model checkpoint at studio-ousia/luke-base and are newly initialized: ['classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
metrics.f1_score([1.0, 0.0, 1.0], [0.0, 0.0, 1.0], average='micro')

0.6666666666666666

Let's verify a forward pass on a batch:

In [20]:
batch = next(iter(valid_dataloader))
labels = batch["label"]
batch.keys()

dict_keys(['input_ids', 'entity_ids', 'entity_position_ids', 'attention_mask', 'entity_attention_mask', 'label'])

In [21]:
batch["input_ids"].shape

torch.Size([2, 512])

In [22]:
del batch["label"]
outputs = model(**batch)
outputs

EntityPairClassificationOutput([('logits',
                                 tensor([[ 0.7272,  0.0416,  0.2963, -0.0381,  0.0115, -0.2739],
                                         [ 0.6918, -0.1223,  0.2337,  0.0159,  0.0440, -0.1138]],
                                        grad_fn=<MmBackward0>))])

In [23]:
labels.float()

tensor([[0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0.]])

Calculate intial loss:

In [24]:
criterion = torch.nn.BCEWithLogitsLoss()

initial_loss = criterion(outputs.logits, labels.float())
print("Initial loss:", initial_loss)

Initial loss: tensor(0.7722, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


## Train the model

Let's train the model. We also use early stopping, to avoid overfitting the training dataset. We also log everything to Weights and Biases, which will give us beautiful charts of the loss and accuracy plotted over time.

If you haven't already, you can create an account on the [website](https://wandb.ai/site), then log in in a web browser, and run the cell below: 

In [25]:
import wandb

wandb.login()

wandb: Currently logged in as: jakr (use `wandb login --relogin` to force relogin)


True

In [26]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import EarlyStopping

wandb_logger = WandbLogger(name='luke-first-run', project='LUKE')
# for early stopping, see https://pytorch-lightning.readthedocs.io/en/1.0.0/early_stopping.html?highlight=early%20stopping
early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=2,
    strict=False,
    verbose=False,
    mode='min'
)

trainer = Trainer(gpus=1, logger=wandb_logger, callbacks=[EarlyStopping(monitor='validation_loss')])
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                            | Params
----------------------------------------------------------
0 | model | LukeForEntityPairClassification | 274 M 
----------------------------------------------------------
274 M     Trainable params
0         Non-trainable params
274 M     Total params
1,098.045 Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/luke-first-run/version_None/checkpoints exists and 

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [1]:
trainer.test()

NameError: ignored

## Evaluation

Instead of calling `trainer.test()`, we can also manually evaluate the model on the entire test set:

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

loaded_model.model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

predictions_total = []
labels_total = []
for batch in tqdm(test_dataloader):
    # get the inputs;
    labels = batch["label"]
    del batch["label"]

    # move everything to the GPU
    for k,v in batch.items():
      batch[k] = batch[k].to(device)

    # forward pass
    outputs = loaded_model.model(**batch)
    logits = outputs.logits
    predictions = logits.argmax(-1)
    predictions_total.extend(predictions.tolist())
    labels_total.extend(labels.tolist())

In [ ]:
print("Accuracy on test set:", accuracy_score(labels_total, predictions_total))

## Inference

Here we test the trained model on a new, unseen sentence.

In [ ]:
loaded_model = LUKE.load_from_checkpoint(checkpoint_path="/content/drive/Shareddrives/Datascouts/epoch=3-step=7699.ckpt")

In [ ]:
test_df.iloc[0].sentence

In [ ]:
import torch.nn.functional as F

idx = 2
text = test_df.iloc[idx].sentence
entity_spans = test_df.iloc[idx].entity_spans  # character-based entity spans 
entity_spans = [tuple(x) for x in entity_spans]

inputs = tokenizer(text, entity_spans=entity_spans, return_tensors="pt")

outputs = loaded_model.model(**inputs)
logits = outputs.logits
predicted_class_idx = logits.argmax(-1).item()
print("Sentence:", text)
print("Ground truth label:", test_df.iloc[idx].string_id)
print("Predicted class idx:", id2label[predicted_class_idx])
print("Confidence:", F.softmax(logits, -1).max().item())